In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import ensemble, preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix, classification_report 

In [33]:
data_train = pd.read_csv('../train.csv' , encoding='big5')
data_test = pd.read_csv('../test.csv' , encoding='big5')

C:\Users\cglog\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
data_train['GENDER'].isnull().sum()

683

In [35]:
data_train['AGE'] = data_train['AGE'].map({'低' : 0, '中' : 1,'中高' : 2, '高' : 3}).astype('int')
data_train['CHARGE_CITY_CD'] = data_train['CHARGE_CITY_CD'].map({'A1' : 0, 'A2' : 1,'B1' : 2, 'B2' : 3, 'C1' : 4, 'C2' : 5, 'D' : 6, 'E' : 7}).astype('int')
data_train['CONTACT_CITY_CD'] = data_train['CONTACT_CITY_CD'].map({'A1' : 0, 'A2' : 1,'B1' : 2, 'B2' : 3, 'C1' : 4, 'C2' : 5, 'D' : 6, 'E' : 7}).astype('int')
data_train['LAST_A_CCONTACT_DT'] = data_train['LAST_A_CCONTACT_DT'].map({'N' : 0, 'Y' : 1}).astype('int')
data_train['LAST_A_ISSUE_DT'] = data_train['LAST_A_ISSUE_DT'].map({'N' : 0, 'Y' : 1}).astype('int')
data_train['LAST_B_ISSUE_DT'] = data_train['LAST_B_ISSUE_DT'].map({'N' : 0, 'Y' : 1}).astype('int')
data_train['Y1'] = data_train['Y1'].map({'N' : 0, 'Y' : 1}).astype('int')


data_test['AGE'] = data_test['AGE'].map({'低' : 0, '中' : 1,'中高' : 2, '高' : 3}).astype('int')
data_test['CHARGE_CITY_CD'] = data_test['CHARGE_CITY_CD'].map({'A1' : 0, 'A2' : 1,'B1' : 2, 'B2' : 3, 'C1' : 4, 'C2' : 5, 'D' : 6, 'E' : 7}).astype('int')
data_test['CONTACT_CITY_CD'] = data_test['CONTACT_CITY_CD'].map({'A1' : 0, 'A2' : 1,'B1' : 2, 'B2' : 3, 'C1' : 4, 'C2' : 5, 'D' : 6, 'E' : 7}).astype('int')
data_test['LAST_A_CCONTACT_DT'] = data_test['LAST_A_CCONTACT_DT'].map({'N' : 0, 'Y' : 1}).astype('int')
data_test['LAST_A_ISSUE_DT'] = data_test['LAST_A_ISSUE_DT'].map({'N' : 0, 'Y' : 1}).astype('int')
data_test['LAST_B_ISSUE_DT'] = data_test['LAST_B_ISSUE_DT'].map({'N' : 0, 'Y' : 1}).astype('int')


In [36]:
data_train['EDUCATION_CD'] = data_train['EDUCATION_CD'].fillna(2)
data_test['EDUCATION_CD'] = data_test['EDUCATION_CD'].fillna(2)

In [37]:
dt = data_train

In [38]:
ohi = pd.get_dummies(dt["GENDER"])
ohi
dt = dt.drop("GENDER",axis=1)
dt = pd.concat([ohi,dt],axis=1)

In [39]:
dt = dt.sample(frac = 1)

fraud_dt = dt.loc[dt['Y1'] == 1]
non_fraud_dt = dt.loc[dt['Y1'] == 0][:2000]

normal_distributed_dt = pd.concat([fraud_dt, non_fraud_dt])

new_dt = normal_distributed_dt.sample(frac=1)
# new_dt = normal_distributed_dt.sample(frac=1, random_state=42)

new_dt.shape

(4000, 133)

In [40]:
new_dt.head()

,F,M,CUS_ID,AGE,CHARGE_CITY_CD,CONTACT_CITY_CD,EDUCATION_CD,MARRIAGE_CD,LAST_A_CCONTACT_DT,L1YR_A_ISSUE_CNT,...,L1YR_GROSS_PRE_AMT,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G,Y1
82416,1,0,1559922,3,2,2,3.0,1.0,1,4,...,0.021465,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
99447,1,0,1030651,3,2,4,2.0,2.0,0,0,...,0.000000,H,Y,N,N,N,N,N,Y,0
9724,0,1,1132351,1,0,0,3.0,1.0,1,0,...,0.000732,G,N,N,N,N,N,N,N,1
16153,1,0,384354,3,0,0,2.0,1.0,0,0,...,0.000122,H,Y,N,Y,N,N,N,Y,1
653,1,0,1142345,0,1,1,2.0,0.0,1,1,...,0.000214,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [41]:
from sklearn import  ensemble, preprocessing, metrics
from sklearn.model_selection import train_test_split

x = new_dt[['AGE','CHARGE_CITY_CD','CONTACT_CITY_CD','EDUCATION_CD','LAST_A_CCONTACT_DT','L1YR_A_ISSUE_CNT','LAST_A_ISSUE_DT','L1YR_B_ISSUE_CNT','LAST_B_ISSUE_DT']]
y = new_dt["Y1"]
train_X, test_X, train_y, test_y = train_test_split(x, y, test_size = 0.333)

forest = ensemble.RandomForestClassifier(n_estimators = 200)
forest_fit = forest.fit(train_X, train_y)

In [42]:
x1 = dt[['AGE','CHARGE_CITY_CD','CONTACT_CITY_CD','EDUCATION_CD','LAST_A_CCONTACT_DT','L1YR_A_ISSUE_CNT','LAST_A_ISSUE_DT','L1YR_B_ISSUE_CNT','LAST_B_ISSUE_DT']]
y1 = dt["Y1"]
test_y_predicted = forest.predict(x1)

# 績效
fpr, tpr, thresholds = metrics.roc_curve(y1, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

0.7099744897959183
